In [0]:
%run /Users/sli0043@safeway.com/sli_sw_connectivity

In [0]:
today = datetime.now(pytz.timezone('America/Boise'))
week_start = today - timedelta(days=(today.weekday()+1)%7)
tst_week_start = week_start - timedelta(days=91)

horizon_begdate = "'" + str(week_start.date()) + "'"
tst_begdate = "'" + str(tst_week_start.date()) + "'"
div='17'
smic=3630
grp=25

print(horizon_begdate)
print(tst_begdate)
print(div)
print(smic)

'2022-11-13'
'2022-08-14'
17
3630

In [0]:
### 1) Download CSC*Store Scope
q=f"""
select distinct
    u_consumer_selling_cd as dmdunit,
    u_store as loc
from 
    scmrep.jda_assortment
where 
    u_consumer_selling_cd in (select distinct dmdunit from scmrep.dmdunit where u_category={smic}) and --- or {grp}
    u_retail_division = {div} and 
    u_primary_upc_sw='Y' and u_primary_orderable_cic='Y' and u_primary_upc_rog<>u_consumer_selling_cd and 
    {today} between u_schem_startdt and u_schem_enddt
"""
pd_table = pd.read_sql(q, get_connector())
spark_table = spark.createDataFrame(pd_table)
save_delta(spark_table, f"promo_scope_{div}_{smic}")


### 2) Upload CSC*Store Scope to EDM
q = f"select * from db_work.promo_scope_{div}_{smic};"
table_name = f"EDM_BIZOPS_PRD.DATA_SCIENCE.promo_scope_{div}_{smic}" ### or {grp}
write_snowflake('edm', q, table_name, "overwrite")

### select * from db_work.promo_scope_33_25;

<command-1978208950638873>:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
 pd_table = pd.read_sql(q, get_connector())
/databricks/spark/python/pyspark/sql/pandas/conversion.py:499: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
 arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]

In [0]:
q = f"""
with upc_scope as (
select distinct 
    upc_nbr as upc_id, 
    consumer_selling_cd as csc_id,
    INTERNET_ITEM_DSC as csc_dsc
from 
    EDM_VIEWS_PRD.DW_VIEWS.D1_UPC
where 
    dw_logical_delete_ind=false and safeway_upc_ind='Y' and 
    lpad(consumer_selling_cd,9,'0') in (select distinct dmdunit from EDM_BIZOPS_PRD.DATA_SCIENCE.promo_scope_{div}_{smic})),

store_scope as 
(select distinct loc as store_id from EDM_BIZOPS_PRD.DATA_SCIENCE.promo_scope_{div}_{smic})

select
    p.csc_dsc,
    p.csc_id,
    p.upc_id,
    sum(o.net_amt+o.mkdn_wod_alloc_amt+o.mkdn_pod_alloc_amt) as rev
from
    EDM_VIEWS_PRD.DW_EDW_VIEWS.txn_facts o,
    upc_scope p,
    store_scope s
where
    o.upc_id=p.upc_id and o.store_id=s.store_id and 
    o.txn_dte between date({horizon_begdate})-7*52*2 and date({horizon_begdate})-1 and
    o.item_qty>=0
group by 1,2,3
order by 4 desc;
""" 
query = read_snowflake("edm", q)
query.display()

csc_dsc,csc_id,upc_id,rev
YOPLAIT LIGHT KEY LIME PIE YOGURT,102054,7047000665,22417.5931
FAGE TOTAL 0% BLENDED CHERRY,1231071,68954408026,21068.9352
YOPLAIT LIGHT CHERRY YOGURT,102046,7047000653,20864.7651
YOPLAIT ORIGINAL STRAWBERRY KIWI,618016,7047000317,19538.8931
O ORGANICS YOGURT WHOLE MILK,784667,7989340744,15694.2974
CHOBANI FLIP LOW-FAT GREEK YOGURT LIMITED BATCH AUTUMN HARVEST CRISP,1131407,81829001776,12146.1195
MOUNTAIN HIGH WHOLE MILK PLAIN YOGURT,102400,7527000160,10788.3152
MOUNTAIN HIGH LOW FAT PLAIN YOGURT,102408,7527000191,8444.0457
YOPLAIT ORIGINAL BOO BERRY AND FRANKEN BERRY LOW FAT YOGURT 8 COUNT,1257124,7047019066,5404.9712
FLIP COOKIES & CREAM,912567,81829001919,4035.4920


In [0]:
q = f"""
with CALENDAR as (
select
    CALENDAR_DT,
    case
        when FISCAL_DAY_WEEK_NBR =1 then CALENDAR_DT
        when FISCAL_DAY_WEEK_NBR =2 then lag(CALENDAR_DT, 1) over(order by CALENDAR_DT)
        when FISCAL_DAY_WEEK_NBR =3 then lag(CALENDAR_DT, 2) over(order by CALENDAR_DT)
        when FISCAL_DAY_WEEK_NBR =4 then lag(CALENDAR_DT, 3) over(order by CALENDAR_DT)
        when FISCAL_DAY_WEEK_NBR =5 then lag(CALENDAR_DT, 4) over(order by CALENDAR_DT)
        when FISCAL_DAY_WEEK_NBR =6 then lag(CALENDAR_DT, 5) over(order by CALENDAR_DT)
        when FISCAL_DAY_WEEK_NBR =7 then lag(CALENDAR_DT, 6) over(order by CALENDAR_DT)
    end as week_start
from EDM_VIEWS_PRD.DW_VIEWS.CALENDAR
where CALENDAR_DT between date({horizon_begdate})-7*(52*3+13) and date({horizon_begdate})-1 and 
  DW_CURRENT_VERSION_IND='TRUE' and DW_LOGICAL_DELETE_IND='FALSE'
order by CALENDAR_DT),

upc_scope as (
select distinct 
    upc_nbr as upc_id, consumer_selling_cd as csc_id
from 
    EDM_VIEWS_PRD.DW_VIEWS.D1_UPC
where 
    dw_logical_delete_ind=false and safeway_upc_ind='Y' and 
  --consumer_selling_cd in (450314,429760,490077,450983,101874,513142,864886,547366,101875,962518)),
  --consumer_selling_cd in (102271,102272)),
    consumer_selling_cd in (1021532,618855)),

store_scope as 
--(select distinct loc as store_id from EDM_BIZOPS_PRD.DATA_SCIENCE.promo_scope_{div}_{smic})
(select distinct facility_nbr as store_id from EDM_VIEWS_PRD.DW_VIEWS.RETAIL_STORE where division_id={div} and dw_current_version_ind=true and dw_logical_delete_ind=false and corporation_id='001')

select
    p.csc_id,
    c.week_start,
    sum(o.item_qty) as qty,
    sum(o.net_amt+o.mkdn_wod_alloc_amt+o.mkdn_pod_alloc_amt)/sum(o.item_qty) as promo_price,
    avg(case when reg_rtl_prc_fctr=0 then NULL else REG_RTL_PRC/reg_rtl_prc_fctr end) as reg_price
from
    EDM_VIEWS_PRD.DW_EDW_VIEWS.txn_facts o,
    upc_scope p,
    store_scope s,
    CALENDAR c
where
    o.upc_id=p.upc_id and o.store_id=s.store_id and o.txn_dte=c.CALENDAR_DT and
    o.item_qty>=0
group by 1,2
having qty>0
order by 1,2;
"""
query = read_snowflake("edm", q)
## save_delta(query, f'promo_hist_{div}_{smic}_top10')
save_delta(query, f'promo_hist_{div}')

In [0]:
q = f"""
with CALENDAR as (
select
   Calendar_Dt,
   case
       when Fiscal_Day_Week_Nbr=1 then Calendar_Dt
       when Fiscal_Day_Week_Nbr=2 then lag(Calendar_Dt, 1) over(order by Calendar_Dt)
       when Fiscal_Day_Week_Nbr=3 then lag(Calendar_Dt, 2) over(order by Calendar_Dt)
       when Fiscal_Day_Week_Nbr=4 then lag(Calendar_Dt, 3) over(order by Calendar_Dt)
       when Fiscal_Day_Week_Nbr=5 then lag(Calendar_Dt, 4) over(order by Calendar_Dt)
       when Fiscal_Day_Week_Nbr=6 then lag(Calendar_Dt, 5) over(order by Calendar_Dt)
       when Fiscal_Day_Week_Nbr=7 then lag(Calendar_Dt, 6) over(order by Calendar_Dt)
   end as ds
from EDM_VIEWS_PRD.DW_VIEWS.CALENDAR
where Calendar_Dt between date({horizon_begdate})-7*(52*3+13) and date({horizon_begdate})-1
and DW_CURRENT_VERSION_IND='TRUE' and DW_LOGICAL_DELETE_IND='FALSE'
order by Calendar_Dt),


REGPRICE as (
select distinct
   p.upc_nbr, 
   f.FACILITY_NBR, 
   c.Calendar_Dt, 
   c.ds,
   ITEM_PRICE_AMT /(case when ITEM_PRICE_FCTR=0 then 1 else ITEM_PRICE_FCTR end) as reg_price
from
   CALENDAR as c
inner join
   EDM_VIEWS_PRD.DW_VIEWS.RETAIL_STORE_UPC_PRICE as p on c.Calendar_Dt>=p.dw_first_effective_dt and c.Calendar_Dt<=p.dw_last_effective_dt
inner join 
   EDM_VIEWS_PRD.DW_VIEWS.FACILITY as f on p.FACILITY_INTEGRATION_ID=f.FACILITY_INTEGRATION_ID
inner join 
    (select distinct upc_nbr, loc as store_id from EDM_BIZOPS_PRD.DATA_SCIENCE.promo_scope_{div}_{smic} a, EDM_VIEWS_PRD.DW_VIEWS.D1_UPC b where a.dmdunit=b.consumer_selling_cd and b.dw_logical_delete_ind=false and b.safeway_upc_ind='Y') s on p.upc_nbr=s.upc_nbr and f.FACILITY_NBR=s.store_id 
where
   p.DW_CURRENT_VERSION_IND=TRUE and p.DW_LOGICAL_DELETE_IND=FALSE and p.ITEM_PRICE_AMT>0 and
   f.DW_CURRENT_VERSION_IND=TRUE and f.DW_LOGICAL_DELETE_IND=FALSE and f.corporation_id='001'),


PROMOPRICE as (
select
   p.upc_nbr, 
   f.FACILITY_NBR, 
   c.Calendar_Dt, 
   c.ds,
   p.COUPON_AMT as promo_price
from
   CALENDAR as c
inner join
   EDM_VIEWS_PRD.DW_VIEWS.PROMOTION_STORE as p on c.Calendar_Dt>=p.PROMOTION_START_DT and c.Calendar_Dt<=p.PROMOTION_END_DT
inner join 
   EDM_VIEWS_PRD.DW_VIEWS.FACILITY as f on p.FACILITY_INTEGRATION_ID=f.FACILITY_INTEGRATION_ID
inner join 
   (select distinct upc_nbr, loc as store_id from EDM_BIZOPS_PRD.DATA_SCIENCE.promo_scope_{div}_{smic} a, EDM_VIEWS_PRD.DW_VIEWS.D1_UPC b where a.dmdunit=b.consumer_selling_cd and b.dw_logical_delete_ind=false and b.safeway_upc_ind='Y') s on p.upc_nbr=s.upc_nbr and f.FACILITY_NBR=s.store_id 
where
   p.DW_CURRENT_VERSION_IND=TRUE and p.DW_LOGICAL_DELETE_IND=FALSE and p.COUPON_AMT>0 and
   f.DW_CURRENT_VERSION_IND=TRUE and f.DW_LOGICAL_DELETE_IND=FALSE and f.corporation_id='001'

UNION SELECT DISTINCT 
   pupa.upc_nbr, 
   rs.retail_store_facility_nbr, 
   c.Calendar_Dt, 
   c.ds, 
   prog.SUGGESTED_RETAIL_PRC
FROM 
  CALENDAR as c
INNER JOIN 
  edm_views_prd.dw_views.promotion_retail_order_group prog ON c.Calendar_Dt>=prog.PROMOTION_START_DT and c.Calendar_Dt<=prog.PROMOTION_END_DT
INNER JOIN 
  edm_views_prd.dw_views.promotion_upc_price_area pupa ON pupa.promotion_rog_integration_id = prog.promotion_rog_integration_id
INNER JOIN 
  EDM_VIEWS_PRD.DW_VIEWS.CORPORATE_ITEM itm ON itm.CORPORATE_ITEM_INTEGRATION_ID = prog.CORPORATE_ITEM_INTEGRATION_ID
INNER JOIN 
  edm_views_prd.dw_views.RETAIL_STORE_SECTION_PRICE_AREA prc ON pupa.PRICE_AREA_CD = prc.PRICE_AREA_CD AND itm.CORPORATE_SECTION_CD = prc.SECTION_CD
INNER JOIN 
  EDM_VIEWS_PRD.DW_VIEWS.D1_RETAIL_STORE rs ON PROG.ROG_ID = rs.ROG_ID AND rs.facility_integration_id = prc.facility_integration_id
inner join 
  (select distinct upc_nbr, loc as store_id from EDM_BIZOPS_PRD.DATA_SCIENCE.promo_scope_{div}_{smic} a, EDM_VIEWS_PRD.DW_VIEWS.D1_UPC b where a.dmdunit=b.consumer_selling_cd and b.dw_logical_delete_ind=false and b.safeway_upc_ind='Y') s on pupa.upc_nbr=s.upc_nbr and rs.retail_store_facility_nbr=lpad(s.store_id,4,0)
WHERE 
  prog.SUGGESTED_RETAIL_PRC > 0
  AND prc.DW_CURRENT_VERSION_IND = TRUE AND prc.DW_LOGICAL_DELETE_IND = FALSE
  AND itm.DW_CURRENT_VERSION_IND = TRUE AND itm.DW_LOGICAL_DELETE_IND = FALSE
  AND prog.DW_CURRENT_VERSION_IND = TRUE AND prog.DW_LOGICAL_DELETE_IND = FALSE
  AND pupa.DW_CURRENT_VERSION_IND = TRUE AND pupa.DW_LOGICAL_DELETE_IND = FALSE
  AND rs.DW_LOGICAL_DELETE_IND  = FALSE
  AND trim(to_char(rs.retail_store_facility_nbr)) != 'N/A')


select
    {div} as div_id,
    s.dmdunit as csc_id,
    o.ds,
    avg(o.reg_price) as reg_price,
    avg(o.promo_price) as promo_price
from
    (select
        r.upc_nbr,
        r.facility_nbr,
        r.Calendar_Dt,
        r.ds,
        reg_price,
        promo_price
    from
        REGPRICE r
    left join
        PROMOPRICE p
    on
        r.upc_nbr=p.upc_nbr and r.facility_nbr=p.facility_nbr and r.Calendar_Dt =p.Calendar_Dt) as o,
    EDM_VIEWS_PRD.DW_VIEWS.D1_UPC as p,
    EDM_BIZOPS_PRD.DATA_SCIENCE.promo_scope_{div}_{smic} as s
where
    p.dw_logical_delete_ind=false and p.safeway_upc_ind='Y' and
    o.upc_nbr=p.upc_nbr and p.consumer_selling_cd=s.dmdunit and o.facility_nbr=s.loc
    and s.dmdunit in (450314,429760,490077,450983,101874,513142,864886,547366,101875,962518)
group by 1,2,3
order by 1,2,3
"""

query = read_snowflake("edm", q)
save_delta(query, f'promo_price_{div}_{smic}_top10')

In [0]:
### 1) Pull smic/div weekly history(L104W)
q = f"""
with calendar as (
select
    CALENDAR_DT,
    case
        when FISCAL_DAY_WEEK_NBR =1 then CALENDAR_DT
        when FISCAL_DAY_WEEK_NBR =2 then lag(CALENDAR_DT, 1) over(order by CALENDAR_DT)
        when FISCAL_DAY_WEEK_NBR =3 then lag(CALENDAR_DT, 2) over(order by CALENDAR_DT)
        when FISCAL_DAY_WEEK_NBR =4 then lag(CALENDAR_DT, 3) over(order by CALENDAR_DT)
        when FISCAL_DAY_WEEK_NBR =5 then lag(CALENDAR_DT, 4) over(order by CALENDAR_DT)
        when FISCAL_DAY_WEEK_NBR =6 then lag(CALENDAR_DT, 5) over(order by CALENDAR_DT)
        when FISCAL_DAY_WEEK_NBR =7 then lag(CALENDAR_DT, 6) over(order by CALENDAR_DT)
    end as week_start
from EDM_VIEWS_PRD.DW_VIEWS.CALENDAR
where CALENDAR_DT between date({horizon_begdate})-7*(52*3+13) and date({horizon_begdate})-7*(52+13)-1 and 
  DW_CURRENT_VERSION_IND='TRUE' and DW_LOGICAL_DELETE_IND='FALSE'
order by CALENDAR_DT),

upc_scope as (
select distinct 
    upc_nbr as upc_id, consumer_selling_cd as csc_id
from 
    EDM_VIEWS_PRD.DW_VIEWS.D1_UPC
where 
    dw_logical_delete_ind=false and safeway_upc_ind='Y' and 
    lpad(consumer_selling_cd,9,'0') in (select distinct dmdunit from EDM_BIZOPS_PRD.DATA_SCIENCE.promo_scope_{div}_{smic})),

store_scope as 
(select distinct loc as store_id from EDM_BIZOPS_PRD.DATA_SCIENCE.promo_scope_{div}_{smic})

select
    c.week_start,
    sum(o.item_qty) as qty
from
    EDM_VIEWS_PRD.DW_EDW_VIEWS.txn_facts o,
    upc_scope p,
    store_scope s,
    calendar c
where
    o.upc_id=p.upc_id and o.store_id=s.store_id and o.txn_dte=c.CALENDAR_DT and
    o.item_qty>=0
group by 1
order by 1;
"""
query = read_snowflake("edm", q)
save_delta(query, f'promo_si0_{div}_{smic}')


### 2) Create smic/div si
q = f"""
with promo_si1 as (select
  case when period=0 then 52 else period end as period,
  avg(qty) as qty
from 
  (select *, rank() over (order by week_start)%52 as period from db_work.promo_si0_{div}_{smic})
group by 1)

select
  *,
  qty/(select avg(qty) as qty_avg from promo_si1) as si
from
  promo_si1
order by period;
"""
query=spark.sql(q)
save_delta(query, f'promo_si_{div}_{smic}')

In [0]:
### Generate PySpark DataFrame
q = f"""
select 
  o.*, r.period, s.si
from
  db_work.promo_hist_{div}_{smic}_top10 o,
  (select week_start, case when rank() over (order by week_start)%52=0 then 52 else rank() over (order by week_start)%52 end as period from (select distinct week_start from db_work.promo_hist_{div}_{smic}_top10)) r,
  db_work.promo_si_{div}_{smic} s
where
  o.week_start=r.week_start and r.period = s.period;
"""   
hist = spark.sql(q)
decimal_to_double = lambda x: re.sub(r'decimal\(\d+,\d+\)', 'double', x)
hist = hist.select(*[F.col(d[0]).astype(decimal_to_double(d[1])) if 'decimal' in d[1] else F.col(d[0]) for d in hist.dtypes])
hist.createOrReplaceTempView('hist')


### Transpose & Upload hol_sarimax
q = f"""
select
    date(ds) as week_start,
    case when holiday='Christmas Day' then 1 else 0 end as xmas,
    case when holiday='Columbus Day' then 1 else 0 end as columbus,
    case when holiday='Fathers Day' then 1 else 0 end as dad,
    case when holiday='Good Friday' then 1 else 0 end as easter,
    case when holiday='Halloween' then 1 else 0 end as halloween,
    case when holiday='Independence Day' then 1 else 0 end as independence,
    case when holiday='Labor Day' then 1 else 0 end as labor ,
    case when holiday='Martin Luther King Day' then 1 else 0 end as mlk,
    case when holiday='Memorial Day' then 1 else 0 end as memorial,
    case when holiday='Mothers Day' then 1 else 0 end as mom,
    case when holiday='New Years Day' then 1 else 0 end as newyear,
    case when holiday='St Patricks Day' then 1 else 0 end as stpatrick,
    case when holiday='Super Bowl' then 1 else 0 end as superbowl,
    case when holiday='Thanksgiving' then 1 else 0 end as thanks,
    case when holiday='Valentines Day' then 1 else 0 end as valentine,
    case when holiday='Veterans Day' then 1 else 0 end as veteran
from
    db_work.hol
order by 1;
"""
hol = spark.sql(q)
hol.createOrReplaceTempView('hol')
hol_sarimax = hol.toPandas()
hol_sarimax['week_start'] = pd.to_datetime(hol_sarimax['week_start'])
hol_sarimax['xmas'] = pd.to_numeric(hol_sarimax['xmas'], downcast='integer')
hol_sarimax['columbus'] = pd.to_numeric(hol_sarimax['columbus'], downcast='integer')
hol_sarimax['dad'] = pd.to_numeric(hol_sarimax['dad'], downcast='integer')
hol_sarimax['easter'] = pd.to_numeric(hol_sarimax['easter'], downcast='integer')
hol_sarimax['halloween'] = pd.to_numeric(hol_sarimax['halloween'], downcast='integer')
hol_sarimax['independence'] = pd.to_numeric(hol_sarimax['independence'], downcast='integer')
hol_sarimax['labor'] = pd.to_numeric(hol_sarimax['labor'], downcast='integer')
hol_sarimax['mlk'] = pd.to_numeric(hol_sarimax['mlk'], downcast='integer')
hol_sarimax['memorial'] = pd.to_numeric(hol_sarimax['memorial'], downcast='integer')
hol_sarimax['mom'] = pd.to_numeric(hol_sarimax['mom'], downcast='integer')
hol_sarimax['newyear'] = pd.to_numeric(hol_sarimax['newyear'], downcast='integer')
hol_sarimax['stpatrick'] = pd.to_numeric(hol_sarimax['stpatrick'], downcast='integer')
hol_sarimax['superbowl'] = pd.to_numeric(hol_sarimax['superbowl'], downcast='integer')
hol_sarimax['thanks'] = pd.to_numeric(hol_sarimax['thanks'], downcast='integer')
hol_sarimax['valentine'] = pd.to_numeric(hol_sarimax['valentine'], downcast='integer')
hol_sarimax['veteran'] = pd.to_numeric(hol_sarimax['veteran'], downcast='integer')


### Pandas DataFrame Data Manipulation
df=hist.toPandas()
df['csc_id'] = pd.to_numeric(df['csc_id'], downcast='integer')
df['week_start'] = pd.to_datetime(df['week_start'])
df['ln_qty'] = np.log(df['qty'])
df['ln_promo_price'] = np.log(df['promo_price'])
df = df.merge(hol_sarimax, how='left', on='week_start').reset_index()
df = df.drop(columns='index')
df.fillna(0, inplace=True)
df.display()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:499: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
 arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]

csc_id,week_start,qty,promo_price,reg_price,period,si,ln_qty,ln_promo_price,xmas,columbus,dad,easter,halloween,independence,labor,mlk,memorial,mom,newyear,stpatrick,superbowl,thanks,valentine,veteran
101874,2019-07-21T00:00:00.000+0000,1678.0,4.750706615,4.791582069875,1,0.951265,7.425357887027151,1.5582933680353535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101874,2019-07-28T00:00:00.000+0000,1772.0,4.715202088,4.780025412961,2,0.976698,7.4798641311650265,1.5507917758244052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101874,2019-08-04T00:00:00.000+0000,1768.0,4.7226651018,4.772869785082,3,1.057054,7.477604243197589,1.5523732804600245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101874,2019-08-11T00:00:00.000+0000,2471.0,3.8477585998,4.776498091603,4,1.041517,7.81237820598861,1.3474907968958953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101874,2019-08-18T00:00:00.000+0000,2724.0,3.3206292952,4.774962735642,5,0.975249,7.909856667269403,1.2001543117136702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101874,2019-08-25T00:00:00.000+0000,1675.0,4.7726419104,4.789459824443,6,1.067177,7.423568444259167,1.5629000111992613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101874,2019-09-01T00:00:00.000+0000,1693.0,4.7957206143,4.816043068641,7,0.973531,7.4342573821331355,1.567723981569361,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101874,2019-09-08T00:00:00.000+0000,1527.0,4.7858385724,4.802083024463,8,1.021402,7.3310603052186325,1.5656612597907227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101874,2019-09-15T00:00:00.000+0000,1493.0,4.7922042197,4.808891491023,9,1.025281,7.30854279753919,1.5669904767039098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101874,2019-09-22T00:00:00.000+0000,1572.0,4.7450699109,4.805808556925,10,0.959344,7.360103972989152,1.557106165546609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
def demand_analysis(dmdunit):
  global df_output
  from sklearn.linear_model import LinearRegression

  data=df[df.csc_id==dmdunit]
  data['trend'] = np.arange(data.shape[0])+1
  # data['markdown'] = data['reg_price'] - data['promo_price']
  data_train = data[data.week_start<tst_begdate]
  data_test  = data[data.week_start>=tst_begdate]

  X_train = data_train[['ln_promo_price','trend','si','xmas','columbus','dad','easter','halloween','independence','labor','mlk','memorial','mom','newyear','stpatrick','superbowl','thanks','valentine','veteran']]
  y_train = data_train['ln_qty']
  X_test = data_test[['ln_promo_price','trend','si','xmas','columbus','dad','easter','halloween','independence','labor','mlk','memorial','mom','newyear','stpatrick','superbowl','thanks','valentine','veteran']]
  y_test = data_test['ln_qty']

  reg = LinearRegression().fit(X_train, y_train)
  print(reg.coef_)
  print(reg.intercept_)
  print(reg.score(X_train,y_train))

  y_train_hat = reg.predict(X_train)
  y_test_hat = reg.predict(X_test)
  data_train['yhat']=y_train_hat
  data_train['qty_hat']=np.exp(y_train_hat)
  data_test['yhat']=y_test_hat
  data_test['qty_hat']=np.exp(y_test_hat)
  data = data_train.append(data_test)
  # data.display()
    
  df_output = df_output.append(data[['csc_id', 'week_start', 'qty', 'promo_price', 'reg_price', 'qty_hat']])

In [0]:
df_output=df.loc[0:0,['csc_id','week_start','qty','promo_price','reg_price']][0:0].copy()
df_output['qty_hat']=1.0

demand_analysis(450314)
demand_analysis(429760)
demand_analysis(490077)
demand_analysis(450983)
demand_analysis(101874)
demand_analysis(513142)
demand_analysis(864886)
demand_analysis(547366)
demand_analysis(101875)
demand_analysis(962518)

save_delta(spark.createDataFrame(df_output),f'promo_fcst_{div}_{smic}')

<command-2224442219069824>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 data['trend'] = np.arange(data.shape[0])+1
[-1.43975031 0.00203662 0.32794585 -0.24074779 -0.03041946 0.08254503
 0.11571817 -0.02581517 0.0206637 0.01382554 -0.036494 0.06564634
 0.00952017 -0.11286551 -0.0769477 0.05764845 -0.2215019 -0.05970088
 -0.03865785]
9.869499543884292
0.5827490201045301
<command-2224442219069824>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 data_train['yhat']=y_train_hat
<command-2224442219069824>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 data_train['qty_hat']=np.exp(y_train_hat)
<command-2224442219069824>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 data_test['yhat']=y_test_hat
<command-2224442219069824>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 data_test['qty_hat']=np.exp(y_test_hat)
<command-2224442219069824>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 data = data_train.append(data_test)
<command-2224442219069824>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 df_output = df_output.append(data[['csc_id', 'week_start', 'qty', 'promo_price', 'reg_price', 'qty_hat']])
<command-2224442219069824>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 data['trend'] = np.arange(data.shape[0])+1
[-1.95456804 0.00342828 0.46077003 -0.23550608 -0.07276975 0.11052085
 0.11429776 -0.08456412 0.06051488 0.0776573 -0.0221726 0.05344561
 0.15310215 -0.12418611 0.03128306 -0.00737379 -0.23098634 -0.04922696
 -0.05977474]
10.336290949315954
0.638167238738276
<command-2224442219069824>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 data_train['yhat']=y_train_hat
<command-2224442219069824>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 data_train['qty_hat']=np.exp(y_train_hat)
<command-2224442219069824>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

csc_id,week_start,qty,promo_price,reg_price,qty_hat
450314,2019-07-21T00:00:00.000+0000,1903.0,6.1811665791,7.653023679417,1921.7361175804585
450314,2019-07-28T00:00:00.000+0000,1971.0,6.1858158803,7.649224441833,1939.681413227425
450314,2019-08-04T00:00:00.000+0000,1873.0,6.1766207154,7.665213675214,1999.8147397413543
450314,2019-08-11T00:00:00.000+0000,1932.0,6.1763600414,7.664821002387,1993.828451243541
450314,2019-08-18T00:00:00.000+0000,1846.0,6.1613083424,7.653793103448,1961.8224697108874
450314,2019-08-25T00:00:00.000+0000,1773.0,6.1905989848,7.657735728031,2012.2018998972035
450314,2019-09-01T00:00:00.000+0000,1856.0,6.2028144397,7.670469715698,1976.9253431334898
450314,2019-09-08T00:00:00.000+0000,1798.0,6.2088442158,7.655387076136,1981.8961815708972
450314,2019-09-15T00:00:00.000+0000,1776.0,6.1926634009,7.668712220762,1995.9493357181143
450314,2019-09-22T00:00:00.000+0000,1768.0,6.2293997172,7.676046511628,1940.6383205720444


In [0]:
q=f"""
with fcst_horizon as (select * from
  (select 
      a.startdate as fcstdate, 
      b.startdate,
      rank() over (partition by a.startdate order by b.startdate) - 1 as lag 
  from 
      (select distinct week_start_dt as startdate from scmmdl.scm_calendar where calendar_dt = {tst_begdate} order by 1) a, 
      (select distinct week_start_dt as startdate from scmmdl.scm_calendar where calendar_dt between {tst_begdate} and date({horizon_begdate})-1 order by 1) b
  WHERE
      a.startdate<=b.startdate
  order by 1,2)),


fcst_scope as (select distinct
    u_consumer_selling_cd as dmdunit,
    {div} as loc
from 
    scmrep.jda_assortment
where 
    u_consumer_selling_cd*1 in (450314,429760,490077,450983,101874,513142,864886,547366,101875,962518)),


fcst_base as (select
    a.dmdunit,
    a.loc,
    b.fcstdate,
    b.startdate,
    b.lag,
    case
        when b.lag=0 then a.lag0
        when b.lag=1 then a.lag1
        when b.lag=2 then a.lag2
        when b.lag=3 then a.lag3
        when b.lag=4 then a.lag4
        when b.lag=5 then a.lag5
        when b.lag=6 then a.lag6
        when b.lag=7 then a.lag7
        when b.lag=8 then a.lag8
        when b.lag=9 then a.lag9
        when b.lag=10 then a.lag10
        when b.lag=11 then a.lag11
        when b.lag=12 then a.lag12
    end as fcst
from
    (select
        o.dmdunit,
        o.loc,
        o.startdate,
        sum(period1) as lag0,
        sum(period2) as lag1,
        sum(period3) as lag2,
        sum(period4) as lag3,
        sum(period5) as lag4,
        sum(period6) as lag5,
        sum(period7) as lag6,
        sum(period8) as lag7,
        sum(period9) as lag8,
        sum(period10) as lag9,
        sum(period11) as lag10,
        sum(period12) as lag11,
        sum(period13) as lag12
    from
        scmrep.fcstwide_hist o,
        fcst_scope s
    where
        o.dmdunit=s.dmdunit and o.loc=s.loc and
        o.loc='33' and
        o.dmdgroup='POS'
    group by 1,2,3) as a,
    fcst_horizon b
where
    a.startdate=b.fcstdate
order by 1,2,3,4),


fcst_override as (select
    o.dmdunit,
    o.loc,
    c.week_start_dt as startdate,
    sum(qty) as fcst
from
    scmrep.fcstnarrow_hist o,
    scmmdl.scm_calendar c,
    fcst_scope s
where
    o.dmdunit=s.dmdunit and o.loc=s.loc and
    o.loc='33' and
    o.dmdgroup='POS' and
    o.startdate=c.calendar_dt 
group by 1,2,3
order by 1,2,3)


select
    a.dmdunit,
    a.loc,
    a.fcstdate,
    a.startdate,
    a.lag,
    a.fcst+nvl(b.fcst,0) as fcst
from
    fcst_base a
left join
    fcst_override b on a.dmdunit=b.dmdunit and a.loc=b.loc and a.startdate=b.startdate
order by 1,2,3,4
"""

pd_table = pd.read_sql(q, get_connector())
spark_table = spark.createDataFrame(pd_table)
save_delta(spark_table, f'promo_fcst_{div}_{smic}_by')

<command-1978208950640188>:109: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
 pd_table = pd.read_sql(q, get_connector())
/databricks/spark/python/pyspark/sql/pandas/conversion.py:499: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
 arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]

In [0]:
%sql
----450314,429760,490077,450983,101874,513142,864886,547366,101875,962518
select a.*, b.fcst as fcst_by from db_work.promo_fcst_33_3630 a
left join db_work.promo_fcst_33_3630_by b on a.csc_id=b.dmdunit and a.week_start=b.startdate 
where csc_id=450314;

csc_id,week_start,qty,promo_price,reg_price,qty_hat,fcst_by
450314,2020-07-19T00:00:00.000+0000,2512.0,6.5167775478,7.674062059238,1979.82678338489,null
450314,2020-07-26T00:00:00.000+0000,2657.0,6.4095350019,7.683569844789,2048.846219337889,null
450314,2020-08-02T00:00:00.000+0000,2723.0,6.2771563716,7.672491289199,2172.140189564297,null
450314,2020-08-09T00:00:00.000+0000,2417.0,6.531706537,7.671622911695,2045.0505366283858,null
450314,2020-08-16T00:00:00.000+0000,2691.0,6.4288433668,7.668040244969,2051.5196987695767,null
450314,2020-08-23T00:00:00.000+0000,2890.0,6.193253218,7.662712615881,2235.615742536068,null
450314,2020-08-30T00:00:00.000+0000,2676.0,6.2174170404,7.660930745479,2160.276107652684,null
450314,2020-09-06T00:00:00.000+0000,2455.0,6.2017444399,7.665598086124,2237.66103069311,null
450314,2020-09-13T00:00:00.000+0000,2569.0,6.2136290385,7.675740236149,2208.156322762772,null
450314,2020-09-20T00:00:00.000+0000,2651.0,6.2074060732,7.680040196516,2168.4517099115174,null


Output can only be rendered in Databricks

In [0]:
%sql
----450314,429760,490077,450983,101874,513142,864886,547366,101875,962518
----'2022-07-17'

select
  csc_id,
  sum(ape_ds*qty)/sum(qty) as mape_ds,
  sum(ape_by*qty)/sum(qty) as mape_by
from
  (select
    a.*, b.fcst as fcst_by,
    abs(qty_hat-qty)/qty as ape_ds,
    abs(fcst-qty)/qty as ape_by
  from db_work.promo_fcst_33_3630 a
  inner join db_work.promo_fcst_33_3630_by b 
  on a.csc_id=b.dmdunit and a.week_start=b.startdate)
group by 1
order by 1;

csc_id,mape_ds,mape_by
101874,0.05767695040070872,0.09618981960645964
101875,0.12075222931160386,0.09650214278144174
429760,0.07159202342057298,0.06941976066566735
450314,0.057023935437416434,0.06069319416042508
450983,0.05780923606459666,0.046666208076486033
490077,0.06059299315103485,0.07682196755492202
513142,0.12142738290145211,0.14228534564977263
547366,0.1733258518883238,0.03463882958715295
864886,0.17499646442105163,0.11232727579831782
962518,0.06350880780839474,0.1557777560102239


In [0]:
upc=3663201944
csc=618855
div=32

# ### 1) Pull price*qty time series
q=f"select week_start, promo_price as otd_price, qty from db_work.hist_csc_top10_div where csc_id={csc} order by week_start;"
spark.sql(q).display()


### 2) Pull basefcst & totfcst for promo peaks
q=f"select dmdunit, loc, fcstdate, startdate, lag, tothist, basefcst, totfcst from scmrep.fcstperfstatic where dmdunit={csc} and loc={div} and fcstdate in ('2021-01-24','2021-06-13','2021-08-22','2022-01-16','2022-08-21') and lag=0 order by fcstdate;"
pd_table = pd.read_sql(q, get_connector())
spark.createDataFrame(pd_table).display()


### 3) Pull promo details for promo peaks
q=f"select * from scmrep.fcstnarrow_hist where dmdunit=618855 and type=5 and loc = '32' order by startdate;"
pd_table = pd.read_sql(q, get_connector())
spark.createDataFrame(pd_table).display()


### 4) Pull regular prices
q=f"""
with CALENDAR as (select
   Calendar_Dt,
   case
       when Fiscal_Day_Week_Nbr=1 then Calendar_Dt
       when Fiscal_Day_Week_Nbr=2 then lag(Calendar_Dt, 1) over(order by Calendar_Dt)
       when Fiscal_Day_Week_Nbr=3 then lag(Calendar_Dt, 2) over(order by Calendar_Dt)
       when Fiscal_Day_Week_Nbr=4 then lag(Calendar_Dt, 3) over(order by Calendar_Dt)
       when Fiscal_Day_Week_Nbr=5 then lag(Calendar_Dt, 4) over(order by Calendar_Dt)
       when Fiscal_Day_Week_Nbr=6 then lag(Calendar_Dt, 5) over(order by Calendar_Dt)
       when Fiscal_Day_Week_Nbr=7 then lag(Calendar_Dt, 6) over(order by Calendar_Dt)
   end as ds
from EDM_VIEWS_PRD.DW_VIEWS.CALENDAR
where Calendar_Dt between '2020-09-27' and '2022-09-24'
and DW_CURRENT_VERSION_IND='TRUE' and DW_LOGICAL_DELETE_IND='FALSE'
order by Calendar_Dt)

select distinct
   p.upc_nbr, 
   c.ds,
   avg(ITEM_PRICE_AMT /(case when ITEM_PRICE_FCTR=0 then 1 else ITEM_PRICE_FCTR end)) as reg_price
from
   CALENDAR as c
inner join
   EDM_VIEWS_PRD.DW_VIEWS.RETAIL_STORE_UPC_PRICE as p on c.Calendar_Dt>=p.dw_first_effective_dt and c.Calendar_Dt<=p.dw_last_effective_dt
inner join 
   EDM_VIEWS_PRD.DW_VIEWS.FACILITY as f on p.FACILITY_INTEGRATION_ID=f.FACILITY_INTEGRATION_ID
where
   p.upc_nbr={upc} and f.facility_nbr in (select distinct store_id from EDM_BIZOPS_PRD.DATA_SCIENCE.by_lag0_accuracy_scope_32) and
   p.ITEM_PRICE_AMT>0 and
   f.DW_CURRENT_VERSION_IND=TRUE and f.DW_LOGICAL_DELETE_IND=FALSE and f.corporation_id='001'
group by 1,2
order by 1,2;
"""
read_snowflake("edm", q).display()

upc_nbr,ds,reg_price
3663201944,2021-12-12,5.990000000000
3663201944,2021-12-19,5.990000000000
3663201944,2022-01-16,6.275714285714
3663201944,2022-01-23,6.490000000000
3663201944,2022-01-30,6.490000000000
3663201944,2022-05-15,6.490000000000
3663201944,2022-05-22,6.490000000000
3663201944,2022-07-03,6.490000000000
3663201944,2022-07-10,6.490000000000
3663201944,2022-02-06,6.490000000000


In [0]:
%sql
--select distinct csc_Id from db_work.promo_hist_19 union select distinct csc_Id from db_work.promo_hist_17;

select * from db_work.promo_si0_33_3630;

week_start,qty
2019-07-21,409733
2019-07-28,429486
2019-08-04,436805
2019-08-11,400766
2019-08-18,408861
2019-08-25,437819
2019-09-01,429433
2019-09-08,464112
2019-09-15,410313
2019-09-22,400603
